In [1]:
# import lightning.pytorch as pl
import pytorch_lightning as pl
import torch
import torch.nn as nn

In [2]:
import sys
import os
import json
import time
import numpy as np

from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
from umap.umap_ import find_ab_params

sys.path.append("..")
from singleVis.custom_weighted_random_sampler import CustomWeightedRandomSampler
from singleVis.vis_models import vis_models as vmodels
from singleVis.losses import UmapLoss, ReconstructionLoss, SingleVisLoss, LocalTemporalLoss, SmoothnessLoss
from singleVis.edge_dataset import DVIDataHandler, LocalTemporalDataHandler
from singleVis.trainer import SingleVisTrainer, LocalTemporalTrainer
from singleVis.data import NormalDataProvider
from singleVis.spatial_edge_constructor import LocalSpatialTemporalEdgeConstructor, SingleEpochSpatialEdgeConstructor
from singleVis.projector import DVIProjector
from singleVis.eval.evaluator import Evaluator
from singleVis.visualizer import visualizer


/home/xianglin/miniconda3/envs/genvis/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/xianglin/miniconda3/envs/genvis/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/xianglin/miniconda3/envs/genvis/lib/python3.9/site-packages/

In [22]:

class LitInstanceNormAE(pl.LightningModule):

    def __init__(self, encoder_dims, decoder_dims, criterion):
        super(LitInstanceNormAE, self).__init__()
        self.criterion = criterion
  
        assert len(encoder_dims) > 1
        assert len(decoder_dims) > 1
        self.encoder_dims = encoder_dims
        self.decoder_dims = decoder_dims

        # Build Encoder
        modules = list()
        for i in range(0, len(self.encoder_dims)-2):
            modules.append(
                nn.Sequential(
                nn.Linear(self.encoder_dims[i], self.encoder_dims[i+1]),
                nn.InstanceNorm1d(self.encoder_dims[i+1]),
                nn.ReLU(True) 
                )
            )
        modules.append(nn.Linear(self.encoder_dims[-2], self.encoder_dims[-1]))
        self.encoder = nn.Sequential(*modules)

        # Build Decoder
        modules = list()
        for i in range(0, len(self.decoder_dims)-2):
            modules.append(
                nn.Sequential(
                    nn.Linear(self.decoder_dims[i], self.decoder_dims[i+1]),
                    nn.InstanceNorm1d(self.decoder_dims[i+1]),
                    nn.ReLU(True)
                )
                
            )
        modules.append(nn.Linear(self.decoder_dims[-2], self.decoder_dims[-1]))
        self.decoder = nn.Sequential(*modules)
    
    def forward(self, edge_to, edge_from):
        outputs = dict()
        embedding_to = self.encoder(edge_to)
        embedding_from = self.encoder(edge_from)
        recon_to = self.decoder(embedding_to)
        recon_from = self.decoder(embedding_from)
        
        outputs["umap"] = (embedding_to, embedding_from)
        outputs["recon"] = (recon_to, recon_from)

        return outputs
    
    def training_step(self, batch, batch_idx):

        # edge_to, edge_from, a_to, a_from, coeffi_from, embedded_from = batch

        # edge_to = edge_to.to(device=self.DEVICE, dtype=torch.float32)
        # edge_from = edge_from.to(device=self.DEVICE, dtype=torch.float32)
        # a_to = a_to.to(device=self.DEVICE, dtype=torch.float32)
        # a_from = a_from.to(device=self.DEVICE, dtype=torch.float32)
        # coeffi_from = coeffi_from.to(device=self.DEVICE, dtype=torch.bool)
        # embedded_from = embedded_from.to(device=self.DEVICE, dtype=torch.float32)

        # outputs = self.model(edge_to, edge_from)
        # _, _, _, loss = self.criterion(edge_to, edge_from, a_to, a_from, coeffi_from, embedded_from, outputs)
        edge_to, edge_from, a_to, a_from = batch
        edge_to = edge_to.to(dtype=torch.float32)
        edge_from = edge_from.to(dtype=torch.float32)
        a_to = a_to.to(dtype=torch.float32)
        a_from = a_from.to(dtype=torch.float32)

        outputs = self.forward(edge_to, edge_from)
        _, _, loss = self.criterion(edge_to, edge_from, a_to, a_from, outputs)
        return loss
    
    def configure_optimizers(self):
        # Define training parameters
        optimizer = torch.optim.Adam(self.parameters(), lr=.01, weight_decay=1e-5)
        # lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, step_size=4, gamma=.1)
        # return [optimizer], [lr_scheduler]
        return [optimizer]
    

In [4]:
VIS_METHOD = "tDVI"
VIS_MODEL = 'litinAE'
CONTENT_PATH = "/home/xianglin/projects/DVI_data/resnet18_mnist"

In [5]:
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

In [6]:
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]
GPU_ID = config["GPU"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]
EPOCH_NAME = config["EPOCH_NAME"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
# VIS_MODEL = VISUALIZATION_PARAMETER["VIS_MODEL"]
LAMBDA1 = VISUALIZATION_PARAMETER["LAMBDA1"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

VIS_MODEL_NAME = VISUALIZATION_PARAMETER["VIS_MODEL_NAME"]
EVALUATION_NAME = VISUALIZATION_PARAMETER["EVALUATION_NAME"]

In [7]:
# Define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))

In [8]:
# Define data_provider
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, device=DEVICE, classes=CLASSES, epoch_name=EPOCH_NAME, verbose=1)
if PREPROCESS:
    data_provider._meta_data()
    if B_N_EPOCHS >0:
        data_provider._estimate_boundary(LEN//10, l_bound=L_BOUND)

Finish initialization...


In [9]:
# Define Losses
negative_sample_rate = 5
min_dist = .1
_a, _b = find_ab_params(1.0, min_dist)
umap_loss_fn = UmapLoss(negative_sample_rate, _a, _b, repulsion_strength=1.0)
recon_loss_fn = ReconstructionLoss(beta=1.0)
smooth_loss_fn = SmoothnessLoss(margin=0.0)
# Define Criterion
criterion = SingleVisLoss(umap_loss_fn, recon_loss_fn, lambd=LAMBDA1)

In [10]:
# Define visualization models
model = LitInstanceNormAE(ENCODER_DIMS, DECODER_DIMS, criterion)

In [11]:
# Define Projector
projector = DVIProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, epoch_name=EPOCH_NAME, device=DEVICE)

In [20]:
# Define Edge dataset
spatial_cons = SingleEpochSpatialEdgeConstructor(data_provider, EPOCH_START, S_N_EPOCHS, B_N_EPOCHS, N_NEIGHBORS, metric="euclidean")
edge_to, edge_from, probs, feature_vectors, attention = spatial_cons.construct()

dataset = DVIDataHandler(edge_to, edge_from, feature_vectors, attention)

n_samples = int(np.sum(S_N_EPOCHS * probs) // 1)
# chose sampler based on the number of dataset
if len(edge_to) > pow(2,24):
    sampler = CustomWeightedRandomSampler(probs, n_samples, replacement=True)
else:
    sampler = WeightedRandomSampler(probs, n_samples, replacement=True)
edge_loader = DataLoader(dataset, batch_size=1000, sampler=sampler, num_workers=4, prefetch_factor=10)

Wed Jun 28 17:23:11 2023 Building RP forest with 17 trees
Wed Jun 28 17:23:12 2023 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
	Stopping threshold met -- exiting after 4 iterations


In [24]:
# from lightning.pytorch import Trainer
trainer = pl.Trainer(max_epochs=1, devices=[0], profiler='pytorch')
trainer.fit(model=model, train_dataloaders=edge_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name      | Type          | Params
--------------------------------------------
0 | criterion | SingleVisLoss | 0     
1 | encoder   | Sequential    | 174 K 
2 | decoder   | Sequential    | 175 K 
--------------------------------------------
349 K     Trainable params
0         Non-trainable params
349 K     Total params
1.399     Total estimated model params size (MB)


[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15
[rank: 0] Received SIGTERM: 15


Epoch 0:   0%|          | 0/379 [00:00<?, ?it/s] 

ERROR:tornado.general:SEND Error: Host unreachable
